In [198]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

In [199]:
# Define a global variable to store the file path
selected_file_path = None

def browse_image():
    global selected_file_path
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png *.jpg *.jpeg *.gif *.bmp *.ppm *.pgm")])
    if file_path:

        # Display the selected image
        img = Image.open(file_path)
        img = img.resize((200, 200))  # Resize the image to fit your window
        img = ImageTk.PhotoImage(img)
        image_display.config(image=img)
        image_display.image = img  # Keep a reference to avoid garbage collection

        # Set the global variable to the selected file path
        selected_file_path = file_path
        
        # ANALYZING
        if(paddy_or_not()):
            analyze_type()



In [200]:
# Create the main application window
app = tk.Tk()
app.title("Analysing Paddy Cultivation")

# app.geometry("500x250")
description_label = tk.Label(app, text="Description:")
description_label.pack()

desc="We have attempted to address this problem in a stagewise, divide and conquer fashion.\n Firstly, given a satellite view of a geographical area, we aim to notify whether there is scope of paddy cultivation or not. Secondly, we look at the probable areas flagged by our first model. The second model is trained to differentiate between paddy and other crops. Lastly, we have models to detect the health of paddy and the variety of paddy being cultivated."

description_text = tk.Label(app, text=desc, wraplength=450)
description_text.pack()

In [201]:
# Create and pack a button to browse and select an image
browse_button = tk.Button(app, text="Browse Image", command=browse_image)
browse_button.pack()

image_display = tk.Label(app)
image_display.pack()
image_display.place()

In [202]:
predicted_ispaddy=tk.Label(app, text="Is paddy: ")
predicted_ispaddy.pack()

In [203]:
predicted_type=tk.Label(app, text="Analyzed Type: None")
predicted_type.pack()

## Predicting

In [204]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
import numpy as np

### 2. To check if it is paddy

In [205]:
def paddy_or_not():
    global selected_file_path
    model = tf.keras.models.load_model('paddy_or_not.h5')

    image = load_img(selected_file_path, target_size=(224,224))
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    image = tf.expand_dims(image, 0)  # Add a batch dimension

    result = model.predict(image)

    class_indices = {0: 'jute', 1: 'maize', 2: 'rice', 3: 'sugarcane', 4: 'wheat'}
    predicted_class = class_indices[np.argmax(result)]
    if predicted_class == 'rice':
        predicted_ispaddy.config(text="Is paddy: "+"True")
        return True  # If the model predicts the image as rice
    else:
        predicted_ispaddy.config(text="Is paddy: "+"False")
        return False

    return True

### 3. To find the type of paddy

In [206]:
def analyze_type():
    global selected_file_path
    model = tf.keras.models.load_model('paddy-type_classification.h5')

    image = load_img(selected_file_path, target_size=(256,256))
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    image = tf.expand_dims(image, 0)  # Add a batch dimension

    predicted = model.predict(image)

    types= ['Healthy', 'Mild', 'Moderate', 'Severe']
    y_pred = [np.array(pred).argmax() for pred in predicted]

    predicted_type.config(text="Analyzed Type: "+ types[y_pred[0]])

In [207]:
# Start the Tkinter main loop
app.mainloop()

1/1 [==============================] - 0s 62ms/step


In [208]:
print("Selected file path:", selected_file_path)

Selected file path: C:/Users/drshw/Desktop/Megathon '23/Paddy-Onfield-Images/Severe/Severe (7).JPG
